# Excel Logic Chain Audit
## Comprehensive Audit of Price Adjustment Excel Logic

This notebook audits and verifies complex Excel logic chains involving:
- **P2 Date Driver**: Central date-based logic
- **Quarter (Semah) Calculations**: Column R
- **Exchange Rate Lookups**: From Arz sheet
- **Index Lookups**: Based on Date, Category, and Chapter

---

In [ ]:
# Install required packages
!pip install openpyxl pandas arabic-reshaper python-bidi matplotlib plotly --quiet

print("Packages installed successfully!")

In [ ]:
# Import libraries
import os
import re
import json
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field
from enum import Enum
from pathlib import Path

import openpyxl
from openpyxl.utils import get_column_letter, column_index_from_string
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Persian language support
try:
    import arabic_reshaper
    from bidi.algorithm import get_display
    PERSIAN_SUPPORT = True
except ImportError:
    PERSIAN_SUPPORT = False
    print("Warning: Persian support not available")

print("Libraries imported successfully!")

In [ ]:
# Persian text utilities
def reshape_persian(text: str) -> str:
    """Reshape Persian text for proper RTL display."""
    if not PERSIAN_SUPPORT or not text:
        return str(text) if text else ""
    try:
        reshaped = arabic_reshaper.reshape(str(text))
        return get_display(reshaped)
    except Exception:
        return str(text)

def is_persian(text: str) -> bool:
    """Check if text contains Persian characters."""
    if not text:
        return False
    persian_pattern = re.compile(r'[\u0600-\u06FF\u0750-\u077F\uFB50-\uFDFF\uFE70-\uFEFF]')
    return bool(persian_pattern.search(str(text)))

# Jalali date utilities
class JalaliDateParser:
    """Parse Jalali (Persian) dates."""
    
    PERSIAN_DIGITS = '\u06f0\u06f1\u06f2\u06f3\u06f4\u06f5\u06f6\u06f7\u06f8\u06f9'
    LATIN_DIGITS = '0123456789'
    
    @classmethod
    def persian_to_latin(cls, text: str) -> str:
        """Convert Persian digits to Latin."""
        if not text:
            return ""
        result = str(text)
        for p, l in zip(cls.PERSIAN_DIGITS, cls.LATIN_DIGITS):
            result = result.replace(p, l)
        return result
    
    @classmethod
    def parse_date(cls, date_str: str) -> Optional[Tuple[int, int, int]]:
        """Parse Jalali date string into (year, month, day)."""
        if not date_str:
            return None
        date_str = cls.persian_to_latin(str(date_str))
        patterns = [
            r'(\d{4})/(\d{1,2})/(\d{1,2})',
            r'(\d{4})-(\d{1,2})-(\d{1,2})',
        ]
        for pattern in patterns:
            match = re.match(pattern, date_str.strip())
            if match:
                year, month, day = map(int, match.groups())
                return (year, month, day)
        return None

print("Utility functions defined.")

## Load Excel File

Upload your Excel file or specify the path.

In [ ]:
# For Google Colab - Upload file
# from google.colab import files
# uploaded = files.upload()
# FILE_PATH = list(uploaded.keys())[0]

# For local use - specify path
FILE_PATH = 'Price_Adjustment_Automated 19 Claude Final 01 Claude Code.xlsx'

# Load workbooks
wb_formulas = openpyxl.load_workbook(FILE_PATH, data_only=False)
wb_values = openpyxl.load_workbook(FILE_PATH, data_only=True)

print(f"Loaded: {FILE_PATH}")
print(f"Sheets: {wb_formulas.sheetnames}")

## 1. Extract Exchange Rates from Arz Sheet

In [ ]:
def load_exchange_rates(wb) -> pd.DataFrame:
    """Load exchange rates from Arz sheet."""
    arz_name = None
    for name in wb.sheetnames:
        if 'Arz' in name:
            arz_name = name
            break
    
    if not arz_name:
        return pd.DataFrame()
    
    sheet = wb[arz_name]
    data = []
    
    for row in range(2, sheet.max_row + 1):
        date = sheet.cell(row=row, column=1).value
        rate = sheet.cell(row=row, column=2).value
        if date and rate:
            data.append({
                'date': JalaliDateParser.persian_to_latin(str(date)),
                'rate': float(rate) if rate else None
            })
    
    return pd.DataFrame(data)

exchange_rates_df = load_exchange_rates(wb_values)
print(f"Loaded {len(exchange_rates_df)} exchange rates")
exchange_rates_df.tail(10)

In [ ]:
# Plot exchange rate trend
if not exchange_rates_df.empty:
    fig = px.line(
        exchange_rates_df.dropna(),
        x='date',
        y='rate',
        title='Exchange Rate Trend (Euro to Rial)'
    )
    fig.update_xaxes(title='Date (Jalali)')
    fig.update_yaxes(title='Rate (Rial)')
    fig.show()

## 2. Extract Main Data from Sheet 1-2

In [ ]:
def extract_main_data(wb) -> pd.DataFrame:
    """Extract main data from the '1-2' sheet."""
    sheet = wb['1-2']
    
    columns = {
        'B': 'row_num',
        'C': 'contract_row',
        'D': 'chapter',
        'E': 'sub_category',
        'F': 'index',
        'G': 'description',
        'H': 'package_name',
        'I': 'proposal_date',
        'J': 'calc_base_date',
        'K': 'entry_date',
        'L': 'currency_ratio_query',
        'M': 'currency_ratio_used',
        'N': 'f_coefficient',
        'O': 'purchase_rate',
        'P': 'base_rate',
        'Q': 'calc_year',
        'R': 'n_value',
        'S': 'n_reduction',
        'T': 'months_elapsed',
        'W': 'contract_amount',
        'X': 'compensation_amount',
    }
    
    data = []
    for row in range(7, sheet.max_row + 1):
        row_data = {'excel_row': row}
        for col_letter, col_name in columns.items():
            col_idx = column_index_from_string(col_letter)
            row_data[col_name] = sheet.cell(row=row, column=col_idx).value
        data.append(row_data)
    
    return pd.DataFrame(data)

main_data_df = extract_main_data(wb_values)
print(f"Extracted {len(main_data_df)} rows from main sheet")
main_data_df.head()

## 3. Verify Mode Settings (H1 Driver)

In [ ]:
# Get mode settings
sheet = wb_values['1-2']
mode = sheet['H1'].value
base_date = sheet['H2'].value
base_rate = sheet['P7'].value

print("=" * 60)
print("MODE SETTINGS VERIFICATION")
print("=" * 60)
print(f"Mode (H1): {mode}")
print(f"Base Date (H2): {base_date}")
print(f"Base Rate (P7): {base_rate}")

# Expected values based on mode
expected = {
    3: {'date': 'Esfand 1398', 'rate': 405150},
    4: {'date': '1403/07/01', 'rate': 506978}
}

if mode in expected:
    exp = expected[mode]
    date_match = str(base_date) == exp['date']
    rate_match = base_rate == exp['rate']
    
    print(f"\nExpected Date: {exp['date']} - {'PASS' if date_match else 'FAIL'}")
    print(f"Expected Rate: {exp['rate']} - {'PASS' if rate_match else 'FAIL'}")

## 4. Verify Quarter (Semah/N) Calculations

In [ ]:
def verify_quarter_calculations(df: pd.DataFrame) -> pd.DataFrame:
    """Verify N/Semah values based on year."""
    results = []
    
    for _, row in df.iterrows():
        j_value = row['calc_base_date']
        r_value = row['n_value']
        
        if j_value is None:
            continue
        
        parsed = JalaliDateParser.parse_date(str(j_value))
        if not parsed:
            continue
            
        year = parsed[0]
        
        # Expected: 0.05 for years 1402-1403
        expected = 0.05 if year in [1402, 1403] else None
        
        try:
            actual = float(r_value) if r_value else None
            match = abs(actual - expected) < 0.001 if actual and expected else False
        except:
            actual = r_value
            match = False
        
        results.append({
            'row': row['excel_row'],
            'date': j_value,
            'year': year,
            'expected_n': expected,
            'actual_n': actual,
            'status': 'PASS' if match else 'FAIL'
        })
    
    return pd.DataFrame(results)

quarter_results = verify_quarter_calculations(main_data_df)
print("Quarter/N Value Verification:")
quarter_results

In [ ]:
# Summary of quarter calculations
if not quarter_results.empty:
    summary = quarter_results['status'].value_counts()
    
    fig = px.pie(
        values=summary.values,
        names=summary.index,
        title='Quarter/N Value Verification Results',
        color_discrete_map={'PASS': 'green', 'FAIL': 'red'}
    )
    fig.show()

## 5. Verify Exchange Rate Lookups

In [ ]:
def verify_exchange_rates(main_df: pd.DataFrame, rates_df: pd.DataFrame) -> pd.DataFrame:
    """Verify exchange rate lookups."""
    rates_dict = dict(zip(rates_df['date'], rates_df['rate']))
    results = []
    
    for _, row in main_df.iterrows():
        k_value = row['entry_date']
        o_value = row['purchase_rate']
        
        if k_value is None or o_value is None:
            continue
        
        date_str = JalaliDateParser.persian_to_latin(str(k_value))
        expected = rates_dict.get(date_str)
        
        try:
            actual = float(o_value)
            if expected:
                match = abs(actual - expected) <= 1.0
                status = 'PASS' if match else 'FAIL'
            else:
                status = 'WARNING'
        except:
            actual = o_value
            status = 'ERROR'
        
        results.append({
            'row': row['excel_row'],
            'date': date_str,
            'expected_rate': expected,
            'actual_rate': actual,
            'status': status
        })
    
    return pd.DataFrame(results)

rate_results = verify_exchange_rates(main_data_df, exchange_rates_df)
print("Exchange Rate Verification:")
rate_results

In [ ]:
# Visualize rate comparisons
if not rate_results.empty:
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=rate_results['date'],
        y=rate_results['expected_rate'],
        mode='markers+lines',
        name='Expected (Arz)',
        marker=dict(color='blue')
    ))
    
    fig.add_trace(go.Scatter(
        x=rate_results['date'],
        y=rate_results['actual_rate'],
        mode='markers+lines',
        name='Actual (Column O)',
        marker=dict(color='green')
    ))
    
    fig.update_layout(
        title='Exchange Rate Comparison: Expected vs Actual',
        xaxis_title='Date',
        yaxis_title='Rate (Rial)'
    )
    fig.show()

## 6. Category Indices Overview

In [ ]:
def extract_category_sheet(wb, sheet_name: str) -> pd.DataFrame:
    """Extract data from a category sheet."""
    if sheet_name not in wb.sheetnames:
        return pd.DataFrame()
    
    sheet = wb[sheet_name]
    data = []
    
    for row in range(4, min(sheet.max_row + 1, 15)):  # First 10 rows
        chapter = sheet.cell(row=row, column=1).value
        description = sheet.cell(row=row, column=2).value
        
        if chapter:
            row_data = {
                'chapter': chapter,
                'description': description
            }
            
            # Get sample values from columns C-H
            for col in range(3, 9):
                val = sheet.cell(row=row, column=col).value
                row_data[f'col_{col}'] = val
            
            data.append(row_data)
    
    return pd.DataFrame(data)

# Sample from each category sheet
category_sheets = ['\u0631\u0634\u062a\u0647 \u0627\u06cc', '\u0645\u06a9\u0627\u0646\u06cc\u06a9', '\u0627\u0628\u0646\u06cc\u0647']

for sheet_name in wb_values.sheetnames:
    if sheet_name not in ['1-2', 'Arz ', 'Arz']:
        sample = extract_category_sheet(wb_values, sheet_name)
        if not sample.empty:
            print(f"\n{'='*40}")
            print(f"Sheet: {reshape_persian(sheet_name)}")
            print(f"{'='*40}")
            print(sample.head())

## 7. Comprehensive Audit Summary

In [ ]:
# Compile comprehensive summary
print("="*80)
print("COMPREHENSIVE EXCEL LOGIC AUDIT SUMMARY")
print(reshape_persian("\u06af\u0632\u0627\u0631\u0634 \u062c\u0627\u0645\u0639 \u0628\u0627\u0632\u0631\u0633\u06cc \u0645\u0646\u0637\u0642 \u0627\u06a9\u0633\u0644"))
print("="*80)
print(f"\nFile: {FILE_PATH}")
print(f"Timestamp: {datetime.now().isoformat()}")
print(f"\nMode: {mode}")
print(f"Base Date: {base_date}")
print(f"Base Rate: {base_rate}")

print("\n" + "-"*40)
print("QUARTER CALCULATIONS (Column R)")
print("-"*40)
if not quarter_results.empty:
    qr_summary = quarter_results['status'].value_counts()
    for status, count in qr_summary.items():
        print(f"  {status}: {count}")

print("\n" + "-"*40)
print("EXCHANGE RATE LOOKUPS (Column O)")
print("-"*40)
if not rate_results.empty:
    er_summary = rate_results['status'].value_counts()
    for status, count in er_summary.items():
        print(f"  {status}: {count}")

print("\n" + "="*80)

In [ ]:
# Combined visualization
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'pie'}]],
    subplot_titles=['Quarter Calculations', 'Exchange Rate Lookups']
)

if not quarter_results.empty:
    qr_counts = quarter_results['status'].value_counts()
    fig.add_trace(
        go.Pie(labels=qr_counts.index, values=qr_counts.values,
               marker_colors=['green' if x=='PASS' else 'red' for x in qr_counts.index]),
        row=1, col=1
    )

if not rate_results.empty:
    er_counts = rate_results['status'].value_counts()
    color_map = {'PASS': 'green', 'FAIL': 'red', 'WARNING': 'orange', 'ERROR': 'gray'}
    fig.add_trace(
        go.Pie(labels=er_counts.index, values=er_counts.values,
               marker_colors=[color_map.get(x, 'gray') for x in er_counts.index]),
        row=1, col=2
    )

fig.update_layout(title_text='Audit Results Summary')
fig.show()

## 8. Export Results

In [ ]:
# Export results to JSON
audit_results = {
    'file': FILE_PATH,
    'timestamp': datetime.now().isoformat(),
    'mode': mode,
    'base_date': str(base_date),
    'base_rate': base_rate,
    'quarter_results': quarter_results.to_dict('records') if not quarter_results.empty else [],
    'rate_results': rate_results.to_dict('records') if not rate_results.empty else [],
}

output_file = 'audit_results_colab.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(audit_results, f, ensure_ascii=False, indent=2)

print(f"Results exported to: {output_file}")

# For Colab - download the file
# from google.colab import files
# files.download(output_file)